# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [20]:
# import libraries
import re
import pickle

import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import nltk

# punkt is a tokenizer that is used to split text into individual words and sentences.
# wordnet is a lexical database of English words that is used for 
# various NLP tasks such as stemming, lemmatization, and synonym detection.
nltk.download(['punkt', 'wordnet']);

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator,TransformerMixin
import joblib

[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [21]:
# load data from database
engine = create_engine('sqlite:///ETL_Preparation.db')
df = pd.read_sql_table('ETL_Preparation', engine)
X = df['message']
df.head(3)


,id,message,original,genre,"('related',)","('request',)","('offer',)","('aid_related',)","('medical_help',)","('medical_products',)",...,"('aid_centers',)","('other_infrastructure',)","('weather_related',)","('floods',)","('storm',)","('fire',)","('earthquake',)","('cold',)","('other_weather',)","('direct_report',)"
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
Y = df[df.columns[4: ]] # start column 4 to the 'end'
category_names  = Y.columns
category_names

Index(['('related',)', '('request',)', '('offer',)', '('aid_related',)',
       '('medical_help',)', '('medical_products',)', '('search_and_rescue',)',
       '('security',)', '('military',)', '('child_alone',)', '('water',)',
       '('food',)', '('shelter',)', '('clothing',)', '('money',)',
       '('missing_people',)', '('refugees',)', '('death',)', '('other_aid',)',
       '('infrastructure_related',)', '('transport',)', '('buildings',)',
       '('electricity',)', '('tools',)', '('hospitals',)', '('shops',)',
       '('aid_centers',)', '('other_infrastructure',)', '('weather_related',)',
       '('floods',)', '('storm',)', '('fire',)', '('earthquake',)',
       '('cold',)', '('other_weather',)', '('direct_report',)'],
      dtype='object')

### 2. Write a tokenization function to process your text data

In [23]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [24]:
pipeline1 = Pipeline([
	('features', FeatureUnion([
    	('text_pipeline', Pipeline([
        	('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
        	('tfidf_transformer', TfidfTransformer())
        ]))
    ])),
    
	('classifier', MultiOutputClassifier(AdaBoostClassifier()))
])



```
pipeline using scikit-learn library. The pipeline consists of two main components:

- FeatureUnion: This component allows combining multiple feature extraction methods. In this example, it includes only one feature extraction method - text_pipeline.

- MultiOutputClassifier: This is a wrapper for multi-output classification algorithms. In this example, it uses the AdaBoostClassifier algorithm for multi-output classification.

The text_pipeline feature extraction method consists of two sub-components:

- CountVectorizer: This is used to convert a collection of text documents to a matrix of token counts.

- TfidfTransformer: This is used to transform a count matrix to a normalized term-frequency times inverse document-frequency representation.

The pipeline is designed to handle multi-output classification problems, meaning that it can predict multiple target variables at the same time. 
```

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape # ((19662,), (6554,), (19662, 36), (6554, 36))

# 'train' making 'equations'
# pipeline_fitted = pipeline1.fit(X_train, y_train)
# 1754_SatApril2023, about 2 minutes
# 1915_SatApril2023, about 3 minutes

# fit the pipeline and save it to a file
pipeline_fitted = pipeline1.fit(X_train, y_train)
joblib.dump(pipeline_fitted, 'pipeline_fitted.pkl')

/workspaces/disaster-response-pipeline-uda/myenv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


['pipeline_fitted.pkl']

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [16]:
y_prediction_train = pipeline_fitted.predict(X_train)
x_prediction_test = pipeline_fitted.predict(X_test)

In [ ]:
y_prediction_train

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.